# WardrobeForge API Testing Notebook

This notebook tests all the API endpoints for the WardrobeForge backend.


In [2]:
import requests
import json
import time

# Base URL - change this to your server URL
BASE_URL = "http://127.0.0.1:4000"

# For production:
# BASE_URL = "https://wardrobeforge.com"


## 1. Health Check

Test that the server is running.


In [3]:
response = requests.get(f"{BASE_URL}/health")
print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")


Status Code: 200
Response: {'status': 'ok'}


## 2. Get Presets

Fetch all available presets (backgrounds/scenes). Each preset has:
- `id`: Numeric ID to use in generation requests
- `name`: Human-readable name
- `ref_image_background_only`: URL to the background preview image


In [4]:
response = requests.get(f"{BASE_URL}/presets")
print(f"Status Code: {response.status_code}")
presets = response.json()
print(json.dumps(presets, indent=2))

# Store preset IDs for later use
available_preset_ids = [p['id'] for p in presets['presets']]
print(f"\nAvailable preset IDs: {available_preset_ids}")

Status Code: 200
{
  "presets": [
    {
      "id": 1,
      "name": "Nature",
      "ref_image_background_only": "https://lrieofamnwrazrxzjqps.supabase.co/storage/v1/object/public/uploads/preset_reference_images/nature_ref_background_only.jpg"
    }
  ]
}

Available preset IDs: [1]


## 3. Get Sample Generations

Fetch sample generations to display as examples. The settings are returned as preset names (decoded from IDs).


In [5]:
response = requests.get(f"{BASE_URL}/sample_generations")
print(f"Status Code: {response.status_code}")
samples = response.json()
print(json.dumps(samples, indent=2))


Status Code: 200
[
  {
    "inputs": {
      "clothing": "https://lrieofamnwrazrxzjqps.supabase.co/storage/v1/object/public/uploads/2c803ed19f1b4d59aadcb2d4754dd1b7.png",
      "model": "https://lrieofamnwrazrxzjqps.supabase.co/storage/v1/object/public/uploads/a430a27018124f5d877df514563ecfa5.png",
      "settings": [
        "Nature"
      ]
    },
    "outputs": [
      {
        "filepath": null,
        "url": "https://d1q70pf5vjeyhc.cloudfront.net/predictions/4bb58382bb1b44e991da65361eff65b9/1.jpeg"
      },
      {
        "filepath": null,
        "url": "https://d1q70pf5vjeyhc.cloudfront.net/predictions/f6f5800f32494e1d93547ceb7b5f5dfe/1.jpeg"
      },
      {
        "filepath": null,
        "url": "https://d1q70pf5vjeyhc.cloudfront.net/predictions/d236d6094b4e40d1a268c352e647fb86/1.jpeg"
      }
    ]
  }
]


## 4. Generate Request

Submit a generation request with:
- `personImage`: The model/person image file
- `clothingImages`: The clothing image file(s)
- `settings`: JSON array of preset IDs (integers)

This returns immediately with a `run_id` and processes in the background.


In [14]:
# Paths to sample images
model_image_path = "/home/ubuntu/wardrobe-backend/dummy-frontend-and-requests-tester/sample_model.jpg"
clothing_image_path = "/home/ubuntu/wardrobe-backend/dummy-frontend-and-requests-tester/sample_clothing.jpg"

# Preset IDs to use (use IDs from the presets endpoint)
preset_ids = [1]  # Nature preset

# Prepare the request
files = {
    'personImage': ('model.jpg', open(model_image_path, 'rb'), 'image/jpeg'),
    'clothingImages': ('clothing.jpg', open(clothing_image_path, 'rb'), 'image/jpeg'),
}
data = {
    'settings': json.dumps(preset_ids)
}

# Submit the request
response = requests.post(f"{BASE_URL}/generate_request", files=files, data=data)
print(f"Status Code: {response.status_code}")
result = response.json()
print(f"Response: {result}")

# Store the run_id for checking progress
run_id = result.get('run_id')
print(f"\nRun ID: {run_id}")


Status Code: 202
Response: {'run_id': 29}

Run ID: 29


## 5. Check Progress

Poll the progress of one or more generation runs.


In [15]:
# Check progress (use the run_id from the previous cell)
if run_id:
    response = requests.post(
        f"{BASE_URL}/check_progress",
        json={"run_ids": [str(run_id)]}
    )
    print(f"Status Code: {response.status_code}")
    print(json.dumps(response.json(), indent=2))
else:
    print("No run_id available. Run the generate_request cell first.")


Status Code: 200
{
  "progress": {
    "29": {
      "error": null,
      "is_complete": false,
      "progress": 25
    }
  }
}


## 6. Poll Until Complete

Poll the progress endpoint until the generation is complete.


In [16]:
def wait_for_completion(run_id, timeout=120, poll_interval=5):
    """Poll until the run is complete or times out."""
    start_time = time.time()
    
    while time.time() - start_time < timeout:
        response = requests.post(
            f"{BASE_URL}/check_progress",
            json={"run_ids": [str(run_id)]}
        )
        
        if response.status_code != 200:
            print(f"Error checking progress: {response.text}")
            return None
        
        progress_data = response.json()['progress'].get(str(run_id), {})
        progress = progress_data.get('progress', 0)
        is_complete = progress_data.get('is_complete', False)
        error = progress_data.get('error')
        
        print(f"Progress: {progress}% | Complete: {is_complete}")
        
        if is_complete:
            return True
        
        if error and error != 'Run not found in progress tracker':
            print(f"Error: {error}")
            return False
        
        time.sleep(poll_interval)
    
    print("Timeout waiting for completion")
    return False

# Wait for the run to complete
if run_id:
    completed = wait_for_completion(run_id)
    print(f"\nGeneration completed: {completed}")
else:
    print("No run_id available. Run the generate_request cell first.")


Progress: 100% | Complete: True

Generation completed: True


## 7. Get Run Results

Fetch the complete results of a generation run.


In [17]:
if run_id:
    response = requests.get(f"{BASE_URL}/runs/{run_id}")
    print(f"Status Code: {response.status_code}")
    run_data = response.json()
    print(json.dumps(run_data, indent=2))
    
    # Display output URLs
    if run_data.get('outputs'):
        print("\n=== Output Images ===")
        for i, output in enumerate(run_data['outputs']):
            print(f"Output {i+1}: {output.get('url')}")
else:
    print("No run_id available. Run the generate_request cell first.")


Status Code: 200
{
  "error": null,
  "id": 29,
  "inputs": {
    "clothing": "https://lrieofamnwrazrxzjqps.supabase.co/storage/v1/object/public/uploads/7dcb36cb32a2490aa684da7e3aedf9f2.png?",
    "model": "https://lrieofamnwrazrxzjqps.supabase.co/storage/v1/object/public/uploads/a45f43498d6644f1ae70fe904c1ee996.png?",
    "settings": [
      1
    ]
  },
  "intermediate_outputs": {
    "step_analyze": {
      "clothing_style": "worn from the waist down and ends below the knees",
      "clothing_type": "skirt",
      "garment_description": {
        "preserve_color": true,
        "preserve_length_and_shape": true,
        "preserve_texture": true,
        "simulate_realistic_draping": true,
        "source": "image_2",
        "style": "worn from the waist down and ends below the knees",
        "type": "skirt"
      },
      "gender": "female",
      "preset_details": [
        {
          "description": "{\"pose\": \"She is standing lightly in tall grass with a relaxed, open posture

## 8. Display Output Images

Display the generated images inline (requires IPython).


In [ ]:
from IPython.display import Image, display

if run_id:
    response = requests.get(f"{BASE_URL}/runs/{run_id}")
    run_data = response.json()
    
    if run_data.get('outputs'):
        for i, output in enumerate(run_data['outputs']):
            url = output.get('url')
            if url:
                print(f"Output {i+1}:")
                display(Image(url=url, width=400))
    else:
        print("No outputs available yet.")
        if run_data.get('error'):
            print(f"Error: {run_data['error']}")
else:
    print("No run_id available. Run the generate_request cell first.")


# WardrobeForge API Test Notebook

This notebook tests all the API endpoints for the WardrobeForge backend.

**Prerequisites:**
- Flask server running on `http://localhost:4000`
- Sample images: `sample_model.jpg` and `sample_clothing.jpg` in this directory

**Note:** The `/generate_request` endpoint requires Supabase storage to be configured with proper RLS policies. If you get an "Unauthorized" error, you need to:
1. Go to Supabase Dashboard → Storage → uploads bucket → Policies
2. Add an INSERT policy that allows uploads (e.g., allow all authenticated or public uploads)


In [ ]:
import requests
import json
import time

BASE_URL = "http://localhost:4000"


## 1. Health Check

Test that the server is running.


In [ ]:
response = requests.get(f"{BASE_URL}/health")
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")


## 2. Get Settings (Presets)

Fetch all available presets from the database.

Returns:
- `id`: Preset ID (integer) - use this in generate_request
- `name`: Preset name
- `image_url`: Background-only reference image URL


In [ ]:
response = requests.get(f"{BASE_URL}/settings")
print(f"Status: {response.status_code}")
settings = response.json()
print(json.dumps(settings, indent=2))

# Store available preset IDs for later use
available_preset_ids = [s['id'] for s in settings.get('settings', [])]
print(f"\nAvailable preset IDs: {available_preset_ids}")


## 3. Get Sample Generations

Fetch sample generations (runs marked with `is_sample=True`).

Note: Settings are returned as names (decoded from IDs).


In [ ]:
response = requests.get(f"{BASE_URL}/sample_generations")
print(f"Status: {response.status_code}")
samples = response.json()
print(f"Number of samples: {len(samples)}")
print(json.dumps(samples, indent=2))


## 4. Generate Request

Submit a generation request with:
- `personImage`: The model/person image file
- `clothingImages`: The clothing image file(s)
- `settings`: JSON array of preset IDs (integers), e.g., `[1]` or `[1, 2, 3]`

Returns a `run_id` immediately. The generation happens in the background.


In [ ]:
# Use preset ID 1 (Nature)
preset_ids = [1]

# Prepare the files
files = {
    'personImage': ('sample_model.jpg', open('sample_model.jpg', 'rb'), 'image/jpeg'),
    'clothingImages': ('sample_clothing.jpg', open('sample_clothing.jpg', 'rb'), 'image/jpeg')
}

data = {
    'settings': json.dumps(preset_ids)
}

response = requests.post(f"{BASE_URL}/generate_request", files=files, data=data)
print(f"Status: {response.status_code}")
result = response.json()
print(json.dumps(result, indent=2))

# Store run_id for progress checking
run_id = result.get('run_id')
print(f"\nRun ID: {run_id}")
